# Setup

In [1]:
# Weights and Biases
!pip install -q wandb
# Tensorflow
!pip install -q tensorflow

In [2]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow import square, reduce_mean
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import Callback
from tensorflow.math import multiply
from math import log

2023-02-22 10:13:51.038490: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# If running in colab, insert your wandb key here

#import config
#Erlend
#wandb.login(key=config.erlend_key)
# Hjalmar
wandb.login(key=config.hjalmar_key)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vinje (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/hjalmarjacobvinje/.netrc


True

# Load, split and normalize data

### Load data

In [4]:
google_colab = True

if google_colab:
    import tensorflow as tf
    # Pring info
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

    # Code to read csv file into Colaboratory:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "1Ic73MqpS5ACG1p2oJ_R5cTuEhlQRM_Q6"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('2013-2022_wo_lags.csv')  
    df_read = pd.read_csv('2013-2022_wo_lags.csv')
else:
    file = "../data/processed_data/2013-2022_wo_lags.csv"
    df_read = pd.read_csv(file)

display(df_read)

,Unnamed: 0,Quote_date,Price,Underlying_last,Strike,TTM,R
0,0,2013-01-02,361.355,1462.33,1100.0,2,0.07
1,1,2013-01-02,336.600,1462.33,1125.0,2,0.07
2,2,2013-01-02,311.350,1462.33,1150.0,2,0.07
3,3,2013-01-02,286.345,1462.33,1175.0,2,0.07
4,4,2013-01-02,281.345,1462.33,1180.0,2,0.07
...,...,...,...,...,...,...,...
12459173,12459173,2022-12-30,6.950,3839.81,8400.0,1085,4.22
12459174,12459174,2022-12-30,5.950,3839.81,8600.0,1085,4.22
12459175,12459175,2022-12-30,5.450,3839.81,8800.0,1085,4.22
12459176,12459176,2022-12-30,4.500,3839.81,9000.0,1085,4.22


In [5]:
df = df_read
del df_read

# Group the data by Quote Date and calculate the mean for Underlying Price
df_agg = df.groupby('Quote_date').mean().reset_index()

# Values to returns
df_agg["Underlying_return"] = df_agg["Underlying_last"].pct_change()

lags = 150

# Add the Underlying Price Lag column
for i in range(1, lags + 1):
    df_agg['Underlying_' + str(i)] = df_agg['Underlying_return'].shift(i)

df = pd.merge(df, df_agg[['Quote_date', "Underlying_return"] + ['Underlying_' + str(i) for i in range(1, lags + 1)]], on='Quote_date', how='left')

/var/folders/wk/x86_p6511l95p594k6qnb98h0000gn/T/ipykernel_35927/676062849.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_agg['Underlying_' + str(i)] = df_agg['Underlying_return'].shift(i)
/var/folders/wk/x86_p6511l95p594k6qnb98h0000gn/T/ipykernel_35927/676062849.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_agg['Underlying_' + str(i)] = df_agg['Underlying_return'].shift(i)
/var/folders/wk/x86_p6511l95p594k6qnb98h0000gn/T/ipykernel_35927/676062849.py:14: PerformanceWarning: DataFrame is highly fragmented.  Thi

In [ ]:
display(df)

### Format input data

In [7]:
# Format settings
max_timesteps = 150
moneyness = False # Moneyness = True WIP
bs_vars = ['Moneyness', 'TTM', 'R'] if moneyness else ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = ['Underlying_last'] + [f'Underlying_{i}' for i in range (1, max_timesteps)]

# Create train, validation and test set split points
train_start = datetime(2017,1,1) 
val_start = train_start + relativedelta(months=84) 
test_start = val_start + relativedelta(months=12)
test_end = test_start + relativedelta(months=12)
train_start = str(train_start.date())
val_start = str(val_start.date())
test_start = str(test_start.date())
test_end = str(test_end.date())

# Split train and validation data
df_train = df[(df['Quote_date'] >= train_start) & ( df['Quote_date'] < val_start)]
df_val = df[(df['Quote_date'] >= val_start) & ( df['Quote_date'] < test_start)]

del df
# Extract target values
train_y = (df_train['Price']/df_train['Strike']).to_numpy() if moneyness else df_train['Price'].to_numpy()
val_y = (df_val['Price']/df_val['Strike']).to_numpy() if moneyness else df_val['Price'].to_numpy()

# If usining moneyness, extract strike
if moneyness:
    train_strike = df_train['Strike'].to_numpy()
    val_strike = df_val['Strike'].to_numpy()

# Convert dataframes to numpy arrays
train_x = [df_train[underlying_lags].to_numpy(), df_train[bs_vars].to_numpy()]
val_x = [df_val[underlying_lags].to_numpy(), df_val[bs_vars].to_numpy()]

del df_train
del df_val

# Scale features based on training set
underlying_scaler = MinMaxScaler()
train_x[0] = underlying_scaler.fit_transform(train_x[0])
val_x[0] = underlying_scaler.transform(val_x[0])

bs_scaler = MinMaxScaler()
train_x[1] = bs_scaler.fit_transform(train_x[1])
val_x[1] = bs_scaler.transform(val_x[1])

# Shuffle training set
np.random.seed(0)
shuffle = np.random.permutation(len(train_x[0]))
train_x = [train_x[0][shuffle], train_x[1][shuffle]]
train_y = train_y[shuffle]
if moneyness:
    train_strike = train_strike[shuffle]

# Reshape data to fit LSTM
train_x = [train_x[0].reshape(len(train_x[0]), max_timesteps,1), train_x[1]]
val_x = [val_x[0].reshape(len(val_x[0]), max_timesteps, 1), val_x[1]]

print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')

Train shape: (4907245, 90, 1), (4907245, 4)
Val shape: (847486, 90, 1), (847486, 4)


# Model construction

In [8]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    # LSTM layers
    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        return_sequences = False
    ))

    # MLP layers
    layers = Concatenate()([model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x[0])/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Hyperparameter search setup

In [9]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'random',
    'name': 'LSTM-MLP v3.3: unfilted data, 2017-2022, single LSTM',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [4, 8, 16, 32]},
        'MLP_units': {
            'values': [50, 100, 200, 400, 600]},
        'LSTM_timesteps': {
            'values': [10, 20, 40, 60, 90, 150]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'Bn_momentum': {
            'values': [0.1, 0.4, 0.7, 0.99]},
        'Lr': {
            'distribution': 'log_uniform',
            'max': log(0.1), 'min': log(0.0001)},
        'Lr_decay': {
            'distribution': 'log_uniform',
            'max': log(1), 'min': log(0.8)},        
        'Minibatch_size': {
            'value': 4096},
        'Min_delta': {
            'value': 0.01 if moneyness else 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 3 if moneyness else 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing') 
#sweep_id = '98bxt6oq'

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Lr_decay uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.
wandb: WARNING   Violation 2. Lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: sua8kmo3
Sweep URL: https://wandb.ai/avogadro/Deep%20learning%20for%20option%20pricing/sweeps/sua8kmo3


# Run hyperparameter search

In [10]:
#WIP
class MSE_LossCallback(Callback):
    def __init__(self, train_x, train_y, train_strike, val_x, val_y, val_strike):
        self.train_x = train_x
        self.train_y = train_y
        self.train_strike = train_strike
        self.val_x = val_x
        self.val_y = val_y
        self.val_strike = val_strike
    
    def on_epoch_end(self, epoch, logs={}):
        train_pred = self.model(train_x)
        val_pred = self.model(val_x)

        train_mse = reduce_mean(square(multiply(train_pred[:,0] - self.train_y, self.train_strike)))
        val_mse = reduce_mean(square(multiply(val_pred[:,0] - self.val_y, self.val_strike)))

        print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')


In [11]:
# Calculate the training and validation MSE loss on the actual option price when using price/strike as the target
def MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike):
    train_pred = model(train_x)
    val_pred = model(val_x)

    train_mse = reduce_mean(square((train_pred[:,0] - train_y)*train_strike))
    val_mse = reduce_mean(square((val_pred[:,0] - val_y)*val_strike))

    print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')

In [12]:
import gc
from tensorflow.keras import backend as k

class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

In [13]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )

        if moneyness:
            mse_callback = MSE_LossCallback(train_x, train_y, train_strike, val_x, val_y, val_strike)

        # Adapt sequence length to config
        train_x_adjusted = [train_x[0][:, :config.LSTM_timesteps, :], train_x[1]]
        val_x_adjusted = [val_x[0][:, :config.LSTM_timesteps, :], val_x[1]]
        print(f'Train shape: {train_x_adjusted[0].shape}, {train_x_adjusted[0].shape}')
        print(f'Val shape: {val_x_adjusted[0].shape}, {val_x_adjusted[0].shape}')

        # Train model
        model.fit(
            train_x_adjusted,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x_adjusted, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback, mse_callback, ClearMemory()] if moneyness else [early_stopping, wandb_callback],
        )

        if moneyness:
            MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike)

### Run full sweep

In [15]:
wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing', count = 100)

wandb: Agent Starting Run: 5659veis with config:
wandb: 	Bn_momentum: 0.7
wandb: 	LSTM_layers: 6
wandb: 	LSTM_timesteps: 20
wandb: 	LSTM_units: 16
wandb: 	Lr: 0.0003907476538068363
wandb: 	Lr_decay: 0.998004453429634
wandb: 	MLP_layers: 5
wandb: 	MLP_units: 50
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 4096
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


2023-02-22 10:30:28.350560: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 16)           11712       ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 20)           0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

### Single run

In [ ]:
config = {
    'LSTM_units': 8,
    'MLP_units': 100,
    'LSTM_timesteps': 20,
    'LSTM_layers': 3,
    'MLP_layers': 4,
    'Bn_momentum': 0.99,
    'Lr': 0.01,
    'Lr_decay': 1,
    'Minibatch_size': 4096,
    'Min_delta': 0.01 if moneyness else 1,
    'Patience': 20,
    'Num_features': 3 if moneyness else 4, 
    'Architecture': 'LSTM-MLP v.0.1',
    'Dataset': f'Moneyness. Train_start: {train_start}, val_start: {val_start}, test_start: {test_start}',
}
trainer(config = config, project = 'Deep learning for option pricing - test area')